In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
data_dir = "path/to/your/data"  # Путь к директории с изображениями
batch_size = 32
num_epochs = 10
learning_rate = 0.001

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 класса: "является" и "не является"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    # Обучение
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Валидация
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model = models.resnet34()
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("model.pth"))
model.eval()

# Классификация нового изображения
image = ...  # Загрузка и преобразование изображения
output = model(image)
_, predicted = torch.max(output.data, 1)

if predicted.item() == 0:
    print("Изображение относится к классу 'является'")
else:
    print("Изображение относится к классу 'не является'")

In [5]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2024-04-21 22:32:50--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.61.25, 3.5.8.131, 54.231.196.161, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.61.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241,19M  1,64MB/s    in 2m 59s  

2024-04-21 22:35:49 (1,35 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [21]:
!unzip -d ./coco-data/ annotations_trainval2017.zip

Archive:  annotations_trainval2017.zip
  inflating: ./coco-data/annotations/instances_train2017.json  
  inflating: ./coco-data/annotations/instances_val2017.json  
  inflating: ./coco-data/annotations/captions_train2017.json  
  inflating: ./coco-data/annotations/captions_val2017.json  
  inflating: ./coco-data/annotations/person_keypoints_train2017.json  
  inflating: ./coco-data/annotations/person_keypoints_val2017.json  


In [17]:
import os
import torch
import torchvision
from pycocotools.coco import COCO

# Пути к данным
data_dir = "./coco-data"
train_annotations = os.path.join(data_dir, "annotations/instances_train2017.json")
val_annotations = os.path.join(data_dir, "annotations/instances_val2017.json")

# Определенные категории (измените на нужные вам)
categories = ["person", "car", "bicycle"]

# Преобразование изображений
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Класс для загрузки COCO dataset
class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, categories, transform=None):
        self.root = root
        self.coco = COCO(annotation)
        self.categories = categories
        self.cat_ids = self.coco.getCatIds(catNms=categories)
        self.img_ids = self.coco.getImgIds(catIds=self.cat_ids)
        self.transform = transform

    def __getitem__(self, index):
        img_id = self.img_ids[index]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root, img_info['file_name'])
        img = torchvision.io.read_image(img_path)
        ann_ids = self.coco.getAnnIds(imgIds=img_id, catIds=self.cat_ids)
        target = self.coco.loadAnns(ann_ids)

        # Добавьте необходимую обработку аннотаций (bounding boxes, masks etc.)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.img_ids)

# Создание датасетов
train_dataset = CocoDataset(os.path.join(data_dir, "train2017"), train_annotations, transform)
val_dataset = CocoDataset(os.path.join(data_dir, "val2017"), val_annotations, transform)

# Создание data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

loading annotations into memory...
Done (t=22.59s)
creating index...
index created!
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!


In [18]:
len(train_dataset), len(val_dataset)

(118287, 5000)

In [19]:
val_dataset[0].shape

RuntimeError: [Errno 2] No such file or directory: './val2017/000000397133.jpg'